In [2]:
def query_to_df(query, graph):
    print("Starting query...", end=" ")
    query_start_time = time.time()
    df = graph.run(query).to_data_frame()
    print("Done ({:.2f} minutes).".format((time.time()-query_start_time)/60))
    return df

In [3]:
# graph = Graph( "bolt://matlaber10.media.mit.edu:7687", auth=('neo4j','myneo'))
graph = Graph( "bolt://neo4j-magtwo:7687", auth=('neo4j','myneo'))
# print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
#     graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
#     graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

### Get Titles

In [2]:
def gen_query(venue):
    query =  """
    MATCH (a:Author)-[:AUTHORED]-(q:Quanta)
    WHERE q.venue = '{}'
    RETURN COLLECT(q.title) AS titles
    """.format(venue)
    return query

def gen_query(venue):
    query =  "MATCH (q:Quanta)-[:PUBLISHED_IN_VENUE]->(v:Venue) WHERE v.name = '{}' RETURN collect(q.title) as titles".format(venue)
    return query

In [41]:
venues = top_42

for venue in venues:
    query = gen_query(venue)
    df = query_to_df(query, graph)
#     df.to_csv('Train_/{}_titles.csv'.format(venue), index = False, encoding = "UTF-8")
    df.to_csv('/tmp/data/titles/Train/{}_titles.csv'.format(venue), index = False, encoding = "UTF-8")

Starting query... Done (0.58 minutes).
Starting query... Done (1.60 minutes).
Starting query... Done (0.10 minutes).
Starting query... Done (1.84 minutes).
Starting query... Done (1.58 minutes).


In [ ]:
import ast
venues = top_42

for venue in venues:
    df_titles = pd.read_csv('/tmp/data/titles/Train/{}_titles.csv'.format(venue))
    title_list = df_titles.iloc[0]['titles']
    texts = ast.literal_eval(title_list)
    print('Number of titles in ', venue, ':', len(texts))

### Generate Titles Across All Journals

In [1]:
import time
import pandas as pd
from py2neo import Graph, Node, Relationship
from textgenrnn import textgenrnn

Using TensorFlow backend.
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Instructions for updating:
Colocations handled automatically by placer.


In [9]:
top_5 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science']
top_10 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science', 'Journal of the American Chemical Society', 'JAMA', 'The New England Journal of Medicine', 'Nature Genetics', 'Neuron']
top_42 = ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']

In [ ]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [7]:
import ast
def gen_titles_concat(venues, n_gen=10, num_epochs_=10, batch_size_=128, rnn_size_=128, rnn_layers_=2):
    texts = []
    for venue in venues: 
        print('Adding Venue: ', venue)
        df_titles = pd.read_csv('~/data/titles/Train/{}_titles.csv'.format(venue))    
        title_list = df_titles.iloc[0]['titles']
        titles = ast.literal_eval(title_list)
        texts.extend(titles)
        print('Number of titles for', venue, ':', len(titles))

    print('Total number of titles:', len(texts))
    
    textgen = textgenrnn()
    name_ = str(rnn_size_) + "_" + str(rnn_layers_)
    textgen.train_on_texts(texts, num_epochs=num_epochs_, gen_epochs=1, new_model=True,
                           save_epochs=1, batch_size=batch_size_, verbose=1, word_level=True,
                          name=name_, rnn_size=rnn_size_, rnn_layers=rnn_layers_)
    textgen.generate_to_file('/u/skokada/data/titles/Gen/{}_gen.txt'.format(name_),
                            n=n_gen, 
                            temperature=[i/n_gen for i in range(n_gen)],
                            max_gen_length=100,
                            progress=False)
    

591660

In [ ]:
import sys
venues = top_42
for rnn_size_ in [32, 64, 128, 256]:
    for rnn_layers_ in [2, 3]:
        name = str(rnn_size_) + "_" + str(rnn_layers_)
        oldStdout = sys.stdout
        file = open('/u/skokada/data/titles/Gen/{}_log.txt'.format(name),'w')
        sys.stdout = file
        gen_titles_concat(venues, rnn_size_ = rnn_size_, rnn_layers_ = rnn_layers_)
        file.close()
        sys.stdout = oldStdout

### Check Titles at Each Epoch


In [ ]:
from textgenrnn import textgenrnn

In [ ]:
ids = []
for rnn_size_ in [32, 64, 128, 256]:
    for rnn_layers_ in [2, 3]:
        name = str(rnn_size_) + "_" + str(rnn_layers_)
        ids.append(name)
for id in ids:
    with open('/u/skokada/data/titles/Gen/{}/{}_titles.txt'.format(id, id), 'w') as f:
        for epoch in range(1, 10):
            textgen = textgenrnn(weights_path = '/u/skokada/data/titles/Gen/{}/{}_weights_epoch_{}.hdf5'.format(id, id, epoch),
                                vocab_path = '/u/skokada/data/titles/Gen/{}/{}_vocab.json'.format(id, id),
                                config_path = '/u/skokada/data/titles/Gen/{}/{}_config.json'.format(id, id))
            temperatures=[i/10 for i in range(1, 11)]
            f.write('Epoch: ' + str(epoch) + '\n')
            for t in temperatures:
                f.write("Temperature: " + str(t) + '\n')
                titles = textgen.generate(n=3, temperature=t, progress=False, return_as_list=True)
                for title in titles:
                    f.write(title + '\n')
                f.write('\n')

### Generate Individual Journal Titles

In [ ]:
import sys
venues = top_42
for venue in venues:
    oldStdout = sys.stdout
    file = open('/u/skokada/data/titles/Gen/{}_log.txt'.format(venue), 'w')
    sys.stdout = file
    gen_titles(venue)
    sys.stdout = oldStdout

In [ ]:
import ast
def gen_titles(venue, n_gen=20, epochs=20, batch=128):
#     df_titles = pd.read_csv('Train/{}_titles.csv'.format(venue))
#     df_titles = pd.read_csv('/tmp/data/titles/Train/{}_titles.csv'.format(venue))
    df_titles = pd.read_csv('~/data/titles/Train/{}_titles.csv'.format(venue))    
    title_list = df_titles.iloc[0]['titles']
    texts = ast.literal_eval(title_list)
    print('Venue: ', venue)
    print('Number of titles:', len(texts))
    textgen = textgenrnn()
    textgen.train_on_texts(texts, num_epochs=epochs, gen_epochs=epochs+1, batch_size=batch, verbose=1)
    textgen.save('/u/skokada/data/titles/Gen/{}.hdf5'.format(venue))
    
#     textgen.generate_to_file('Gen/{}_generated.txt'.format(venue),
#                     n=n_gen, 
#                     temperature=[.2 for i in range(len(texts))],
#                     max_gen_length=100,
#                     progress=False)
    textgen.generate_to_file('/u/skokada/data/titles/Gen/{}_generated.txt'.format(venue),
                            n=n_gen, 
                            temperature=[i/n_gen for i in range(n_gen)],
                            max_gen_length=100,
                            progress=False)
    